# SecretFlow is like React Hooks [bob]


In [ ]:
self_party: str = "bob"
peer_party: str = "alice"
self_ray_port: int = 32401
peer_ray_port: int = 32400
self_secretflow_port: int = 8081
peer_secretflow_port: int = 8080
self_spu_port: int = 8091
peer_spu_port: int = 8090

In [ ]:
from secretnote.compat.secretflow.device.driver import (
    SFConfigNetworked,
    SFClusterConfig,
    SFClusterParty,
)

secretflow_config = SFConfigNetworked(
    address=f"localhost:{self_ray_port}",
    cluster_config=SFClusterConfig(
        parties={
            self_party: SFClusterParty(address=f"127.0.0.1:{self_secretflow_port}"),
            peer_party: SFClusterParty(address=f"127.0.0.1:{peer_secretflow_port}"),
        },
        self_party=self_party,
    ),
    logging_level="debug",
)

In [ ]:
from secretnote.compat.spu import (
    SPUConfig,
    SPUClusterDef,
    SPUNode,
    SPURuntimeConfig,
    SPUProtocolKind,
    SPUFieldType,
)

spu_config = SPUConfig(
    cluster_def=SPUClusterDef(
        nodes=[
            SPUNode(party=self_party, address=f"127.0.0.1:{self_spu_port}"),
            SPUNode(party=peer_party, address=f"127.0.0.1:{peer_spu_port}"),
        ],
        runtime_config=SPURuntimeConfig(
            protocol=SPUProtocolKind.SEMI2K,
            field=SPUFieldType.FM128,
        ),
    )
)

In [ ]:
import secretflow

secretflow.shutdown()
secretflow.init(**secretflow_config.dict())

In [ ]:
bob = secretflow.PYU(self_party)
alice = secretflow.PYU(peer_party)
spu = secretflow.SPU(**spu_config.dict())

In [ ]:
import jax.numpy as jnp
import numpy as np
import secretnote.functional as sfx


def multiply(x: np.ndarray, y: float):
    return jnp.multiply(x, y)

In [ ]:
array = sfx.use_cleartext(alice)(...)
array = sfx.use_relocation(spu, array)()

multiplier = sfx.use_cleartext(bob)(0.5)
multiplier = sfx.use_relocation(spu, multiplier)()

result = sfx.use_function(spu, array, multiplier)(multiply)

In [ ]:
result_for_bob = sfx.use_relocation(bob, result)()
result_for_alice = sfx.use_relocation(alice, result)()